In [2]:
import pandas as pd
import urllib.request  
import gzip  
import os  
  
  
def download_and_unzip_file(url, dest_path):  
    # Download the file  
    urllib.request.urlretrieve(url, dest_path + '.gz')  
  
    # Unzip the file  
    with gzip.open(dest_path + '.gz', 'rb') as f_in:  
        # Open the unzipped file in write binary mode  
        with open(dest_path, 'wb') as f_out:  
            # Transfer the contents from the zipped file to the unzipped file  
            f_out.write(f_in.read())  
  
    # Delete the zipped file  
    os.remove(dest_path + '.gz')  

In [3]:
# Update data
download_and_unzip_file('https://datasets.imdbws.com/title.basics.tsv.gz', 'title.basics.tsv')  
download_and_unzip_file('https://datasets.imdbws.com/title.ratings.tsv.gz', 'title.ratings.tsv')  

In [4]:
ratings = pd.read_csv("./title.ratings.tsv", sep="\t")
titles = pd.read_csv("./title.basics.tsv", sep="\t")

/var/folders/x_/0z24g8110_n09vlvhxmrh0_w0000gp/T/ipykernel_53445/2429809888.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  titles = pd.read_csv("./title.basics.tsv", sep="\t")


In [5]:
print(ratings.shape)
ratings.head()

(1401038, 3)


,tconst,averageRating,numVotes
0,tt0000001,5.7,2023
1,tt0000002,5.7,271
2,tt0000003,6.5,1953
3,tt0000004,5.4,178
4,tt0000005,6.2,2723


In [6]:
print(titles.shape)
titles.head()

(10540182, 9)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [7]:
merged_array = pd.merge(ratings, titles, on='tconst', how='inner')  

In [8]:
print(merged_array.shape)
merged_array.head()

(1401038, 11)


,tconst,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,5.7,2023,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,5.7,271,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,6.5,1953,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,5.4,178,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,6.2,2723,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [9]:
merged_array['titleType'].value_counts()

titleType
tvEpisode       695999
movie           305064
short           155344
tvSeries         93177
video            52404
tvMovie          52261
tvMiniSeries     16618
videoGame        15846
tvSpecial        12055
tvShort           2270
Name: count, dtype: int64

In [16]:
filtered_df = merged_array[(merged_array['averageRating'] >=  7.5) & (merged_array['startYear'] == "2023") & (merged_array['numVotes'] > 50000) & (merged_array['titleType'].str.contains('movie|tvMovie'))]  
# filtered_df = merged_array[(merged_array['averageRating'] >=  8) & (merged_array['startYear'] == "2022") & (merged_array['numVotes'] > 50000) & (merged_array['titleType'].str.contains('movie|tvSeries|tvMovie'))]  
sorted_df = filtered_df.sort_values(by=['numVotes', 'averageRating', ], ascending=[False, False])
# sorted_df = filtered_df.sort_values(by=['averageRating', 'numVotes'], ascending=[False, False])

pd.set_option('display.max_rows', 40)  
sorted_df.head(40)[["averageRating","numVotes","originalTitle","titleType","startYear"]]

,averageRating,numVotes,originalTitle,titleType,startYear
734177,8.4,624134,Oppenheimer,movie,2023
1268908,7.9,367326,Guardians of the Galaxy Vol. 3,movie,2023
1383164,8.6,341984,Spider-Man: Across the Spider-Verse,movie,2023
476869,7.7,322381,John Wick: Chapter 4,movie,2023
1390698,7.7,223190,Mission: Impossible - Dead Reckoning Part One,movie,2023
1193632,7.7,191505,Killers of the Flower Moon,movie,2023
1152589,7.5,153173,The Covenant,movie,2023
1307396,7.7,105744,Sound of Freedom,movie,2023
939611,9.2,93579,12th Fail,movie,2023
765399,7.9,86756,La sociedad de la nieve,movie,2023


In [11]:
filtered_df = merged_array[(merged_array['averageRating'] >=  8) & (merged_array['startYear'] == "2022") & (merged_array['numVotes'] > 100000) & (merged_array['titleType'].str.contains('movie|tvSeries|tvMovie'))]  
sorted_df_votes = filtered_df.sort_values(by=['numVotes'], ascending=[False])
sorted_df_votes.head(20)[["averageRating","numVotes","originalTitle","titleType","startYear"]]

,averageRating,numVotes,originalTitle,titleType,startYear
796911,8.3,671304,Top Gun: Maverick,movie,2022
502298,8.6,572769,The Kashmir Files,movie,2022
524855,8.4,359679,House of the Dragon,tvSeries,2022
640012,8.1,350308,Wednesday,tvSeries,2022
1380387,8.1,195024,Reacher,tvSeries,2022
689425,8.6,192491,The Bear,tvSeries,2022
529788,8.7,189465,Severance,tvSeries,2022
1379172,8.4,159651,Andor,tvSeries,2022
495904,8.3,148769,K.G.F: Chapter 2,movie,2022
624129,8.3,131753,Peacemaker,tvSeries,2022


In [12]:
filtered_df = merged_array[(merged_array['averageRating'] >=  5) & (merged_array['startYear'] == "2024") & (merged_array['numVotes'] > 10000) & (merged_array['titleType'].str.contains('movie|tvSeries|tvMovie'))]  
sorted_df = filtered_df.sort_values(by=['averageRating', 'numVotes'], ascending=[False, False])
sorted_df.head(20)[["averageRating","numVotes","originalTitle","titleType","startYear"]]

,averageRating,numVotes,originalTitle,titleType,startYear
736011,8.5,16190,Hanu Man,movie,2024
733869,7.8,18973,Merry Christmas,movie,2024
801597,7.7,16372,The Brothers Sun,tvSeries,2024
1289684,7.7,13324,Hazbin Hotel,tvSeries,2024
658707,7.5,33434,Fighter,movie,2024
730098,6.5,34515,The Beekeeper,movie,2024
555245,6.3,12298,Mean Girls,movie,2024
694992,6.2,17100,Guntur Kaaram,movie,2024
715499,6.1,11473,Argylle,movie,2024
846757,5.7,49413,Indian Police Force,tvSeries,2024
